In [ ]:
using Plots, LaTeXStrings
default(markersize=3, linewidth=1.5)
using Images, TestImages
using MatrixDepot, JLD
using LinearAlgebra
using SparseArrays, IncompleteLU, Arpack
using IterativeSolvers, LinearMaps, Preconditioners
#include("FNC.jl");
include("functions/chapter02.jl")
include("functions/chapter08.jl")

# Example 8.1.1

Here we load the adjacency matrix of a graph with 2790 nodes. Each node is a web page referring to Roswell, NM, and the edges represent links between web pages.

In [ ]:
vars = load("roswelladj.jld")       # get from the book's website

In [ ]:
i, j = vars["i"], vars["j"]
n = maximum([i; j])

In [ ]:
A = sparse(i,j,ones(size(i)),n,n)

We may define the density of $\mathbf{A}$ as the number of nonzeros divided by the total number of entries.

In [ ]:
nnz(A)

In [ ]:
n^2

In [ ]:
density = nnz(A) / n^2

We can compare the storage space needed for the sparse $\mathbf{A}$ with the space needed for its dense or full counterpart. This ratio can never be as small as the density of nonzeros, because of the need to store locations as well as data. However, it's still quite small here, even though the matrix is not really large.

In [ ]:
F = Matrix(A)

In [ ]:
varinfo(r"A")                       # to see memory consumption

In [ ]:
varinfo(r"F")

In [ ]:
storageratio = 111e3/59e6

Matrix-vector products are also much faster using the sparse form, because operations with structural zeros are skipped.

In [ ]:
x = randn(n)
b = A*x
@elapsed for i = 1:200; A*x; end

In [ ]:
b = F*x
@elapsed for i = 1:200; F*x; end

However, the sparse storage format is column-oriented. Operations on rows may take a lot longer than similar ones on columns. (Note: Such behavior is dramatic here for MATLAB, but not Julia.)

In [ ]:
v = A[:,1000]

r = v'

In [ ]:
println("time for replacing columns:")
for i = 1:n; A[:,i]=v; end     # run once to improve timing accuracy
@elapsed for i = 1:n; A[:,i]=v; end

In [ ]:
println("time for replacing rows:")
for i = 1:n; A[i,:]=r; end     # run once to improve timing accuracy
@elapsed for i = 1:n; A[i,:]=r; end

# Example 8.1.2

Here is the adjacency matrix of a graph representing a "small world" network featuring connections to neighbors and a small number of strangers. 

In [ ]:
A = matrixdepot("smallworld",100,4,0.2)

In [ ]:
spy(A,color=:black,size=(600,600),m=1.5,colorbar=false)
xlims!(-5,105); ylims!(-5,105)

The number of vertex pairs connected by a path of length $k>1$ grows with $k$, as can be seen here for $k=4$. (This would be "four degrees of separation."

In [ ]:
spy(A^2,color=:bluesreds,size=(600,600),m=1.5)
xlims!(-5,105); ylims!(-5,105)

# Example 8.1.3

Here is a matrix with both lower and upper bandwidth equal to one. Such a matrix is called *tridiagonal*. The `spdiagm` command creates a sparse matrix given its diagonal elements. The main or central diagonal is numbered zero, above and to the right of that is positive, and below and to the left is negative.

In [ ]:
n = 50;
A = spdiagm(-3=>fill(n,n-3),0=>ones(n),1=>-(1:n-1))

In [ ]:
Matrix( A[1:7,1:7] )

Without pivoting, the LU factors have the same lower and upper bandwidth as the orignal matrix. 

In [ ]:
L,U = lufact(A)
spy(A,layout=(1,3),subplot=1,markersize=2,title="A",color=:bluesreds,colorbar=false,size=(900,300))
spy!(sparse(L),subplot=2,markersize=2,title="L",color=:bluesreds,colorbar=false)
spy!(sparse(U),subplot=3,markersize=2,title="U",color=:bluesreds,colorbar=false)

However, if we introduce row pivoting, bandedness may be expanded or destroyed.

In [ ]:
fact = lu(A)
spy(A,layout=(1,3),subplot=1,markersize=2,title="A",color=:bluesreds,colorbar=false,size=(900,300))
spy!(sparse(fact.L),subplot=2,markersize=2,title="L",color=:bluesreds,colorbar=false)
spy!(sparse(fact.U),subplot=3,markersize=2,title="U",color=:bluesreds,colorbar=false)

# Example 8.1.4

The following generates a random sparse matrix with prescribed eigenvalues.

In [ ]:
n = 1000
density = 1.23e-3
lambda = @. 1/(1:n)
A = sprandsym(n,density,lambda);

In [ ]:
spy(A,title="Sparse symmetric matrix",color=:bluesreds,colorbar=false)

In [ ]:
lambda,V = eigs(A,nev=5,which=:LM)    # largest magnitude
lambda

In [ ]:
lambda,V = eigs(A,nev=5,sigma=0)    # closest to zero
1 ./ lambda

The scaling of time to solve a sparse linear system is not easy to predict unless you have some more information about the matrix (such as bandedness). But it will typically be a great deal faster than the dense or full matrix case.

In [ ]:
x = @. 1/(1:n);  b = A*x;
@time sparse_err = norm(x - A\b)
@time sparse_err = norm(x - A\b)

In [ ]:
A = Matrix(A)  # convert to regular matrix
x = @. 1/(1:n);  b = A*x;
@time dense_err = norm(x - A\b)
@time dense_err = norm(x - A\b)

In [ ]:
0.108273/0.000476

In [ ]:
sparse_err, dense_err

# Example 8.2.1

Here we let $\mathbf{A}$ be a $5\times 5$ matrix. We also choose a random 5-vector.

In [ ]:
A = rand(1.:9.,5,5)

In [ ]:
sum(A,dims=1)

In [ ]:
A = A./sum(A,dims=1)

In [ ]:
sum(A,dims=1)

In [ ]:
eigen(A)

In [ ]:
x = randn(5)

Applying matrix-vector multiplication once doesn't do anything recognizable.

In [ ]:
y = A*x

Repeating the multiplication still doesn't do anything obvious.

In [ ]:
z = A*y

But if we keep repeating the matrix-vector multiplication, something remarkable happens: $\mathbf{A}\mathbf{x}\approx \mathbf{x}$. 

In [ ]:
for j = 1:8;  x = A*x;  end
[x A*x]

In [ ]:
norm(x - A*x)

This seems to occur regardless of the starting value of $x$. 

In [ ]:
x = randn(5)
for j = 1:10;  x = A*x;  end   # computing A^10*x
[x A*x]

In [ ]:
norm(x - A*x)

# Example 8.2.2

We set up a $5\times 5$ matrix with prescribed eigenvalues, then apply the power iteration.

In [ ]:
lambda = [1,-0.75,0.6,-0.4,0]
A = triu(ones(5,5),1) + diagm(lambda)   # triangular matrix, eigs on diagonal

We run the power iteration 60 times. The best estimate of the dominant eigenvalue is the last entry of `gamma`. 

In [ ]:
gamma,x = poweriter(A,60)
gamma

In [ ]:
eigval = gamma[end]

We check linear convergence using a log-linear plot of the error. We use our best estimate in order to compute the error at each step.

In [ ]:
err = eigval .- gamma
plot(0:59,abs.(err),m=:o,label="", 
    title="Convergence of power iteration",
    xlabel=L"k",yaxis=(L"|\lambda_1 - \gamma_k|",:log10,[1e-9,1e1]) )

The trend is clearly a straight line asymptotically. We can get a refined estimate of the error reduction in each step by using the exact eigenvalues.

In [ ]:
@show theory = lambda[2]/lambda[1];
@show observed = err[40]/err[39];

Note that the error is supposed to change sign on each iteration. An effect of these alternating signs is that estimates oscillate around the exact value.

In [ ]:
gamma[36:40]

In [ ]:
lambar = (gamma[60]+gamma[59])/2

# Example 8.3.1

We set up a $5\times 5$ triangular matrix with prescribed eigenvalues on its diagonal.

In [ ]:
lambda = [1,-0.75,0.6,-0.4,0]
A = triu(ones(5,5),1) + diagm(lambda)   # triangular matrix, eigs on diagonal

We run inverse iteration with the shift $s=0.7$ and take the final estimate as our ``exact'' answer to observe the convergence. 

In [ ]:
gamma,x = inviter(A,0.7,30)
gamma

In [ ]:
eigval = gamma[end]

As expected, the eigenvalue that was found is the one closest to $0.7$. The convergence is again linear.

In [ ]:
err = eigval .- gamma
plot(0:29,abs.(err),m=:o,label="", 
    title="Convergence of inverse iteration",
    xlabel=L"k",yaxis=(L"|\lambda_3 - \gamma_k|",:log10,[1e-16,1]) )

The observed linear convergence rate is found from the data. 

In [ ]:
observed_rate = err[22]/err[21]

In the numbering of this example, the eigenvalue closest to $s=0.7$ is $\lambda_3$ and the next-closest is $\lambda_1$.

In [ ]:
lambda

In [ ]:
lambda .- 0.7

In [ ]:
theoretical_rate = (lambda[3]-0.7) / (lambda[1]-0.7)

# Example 8.3.2

In [ ]:
lambda = [1,-0.75,0.6,-0.4,0]
A = triu(ones(5,5),1) + diagm(lambda)   # triangular matrix, eigs on diagonal

We begin with a shift $s=0.7$, which is closest to the eigenvalue 0.6.

In [ ]:
s = 0.7
x = ones(5)
y = (A-s*I)\x
gamma = x[1]/y[1] + s

Note that the result is not yet any closer to the targeted $0.6$. But we proceed (without being too picky about normalization here).

In [ ]:
s = gamma
x = y/y[1]
y = (A-s*I)\x
gamma = x[1]/y[1] + s

Still not much apparent progress. However, in just a few more iterations the results are dramatically better.

In [ ]:
for k = 1:4
    s = gamma  
    x = y/y[1]
    y = (A-s*I)\x  
    gamma = x[1]/y[1] + s
    @show gamma
end

# Example 8.4.1

First we define a triangular matrix with known eigenvalues and a random vector $b$.

In [ ]:
lambda = @. 10 + (1:100)
A = triu(rand(100,100),1) + diagm(lambda)
b = rand(100);
A

Next we build up the first ten Krylov matrices iteratively, using renormalization after each matrix-vector multiplication. 

In [ ]:
Km = [b/norm(b) zeros(100,29)]
for m = 1:29      
    v = A*Km[:,m]
    Km[:,m+1] = v/norm(v)
end
Km

Now we solve a least squares problem for Krylov matrices of increasing dimension.

In [ ]:
resid = zeros(30)
for m = 1:30  
    z = (A*Km[:,1:m])\b  # Solve the least-squares problems
    x = Km[:,1:m]*z
    resid[m] = norm(b-A*x)
end
resid

The linear system approximations show smooth linear convergence at first, but the convergence stagnates after only a few digits have been found.

In [ ]:
plot(0:29,resid,m=:o,
    xaxis=(L"m"),yaxis=(:log10,L"\| b-Ax_m \|"), 
    title="Residual for linear systems",leg=:none)

# Example 8.4.2

We illustrate a few steps of the Arnoldi iteration for a small matrix.

In [ ]:
A = rand(1.:9.,6,6)

The seed vector determines the first member of the orthonormal basis.

In [ ]:
u = randn(6)
Q = u/norm(u)

Multiplication by $\mathbf{A}$ gives us a new vector in $\mathcal{K}_2$. 

In [ ]:
Aq = A*Q[:,1]

We subtract off its projection in the previous direction. The remainder is rescaled to give us the next orthonormal column.

In [ ]:
v = Aq - dot(Q[:,1],Aq)*Q[:,1]
Q = [Q v/norm(v)]

On the next pass, we have to subtract off the projections in two previous directions.

In [ ]:
Aq = A*Q[:,2]
v = Aq - dot(Q[:,1],Aq)*Q[:,1] - dot(Q[:,2],Aq)*Q[:,2]
Q = [Q v/norm(v)]

At every step, $Q_m$ is an ONC matrix.

In [ ]:
opnorm( Q'*Q - I )

And $Q_m$ spans the same space as the 3-dimensional Krylov matrix.

In [ ]:
K = [ u A*u A*A*u ];
[Q K]

In [ ]:
@show rank( [Q K] )

In [ ]:
C = Q\K

In [ ]:
Q*C - K

# Example 8.5.1

We define a triangular matrix with known eigenvalues and a random vector $b$.

In [ ]:
lambda = @. 10 + (1:100)
A = triu(rand(100,100),1) + diagm(lambda)
b = rand(100);

Instead of building up the Krylov matrices, we use the Arnoldi iteration to generate equivalent orthonormal vectors. 

In [ ]:
Q,H = arnoldi(A,b,60);

In [ ]:
H

In [ ]:
Q'Q

In [ ]:
orthogres = [norm(Q[:,1:m]'Q[:,1:m] - I) for m = 1:61]

In [ ]:
plot(0:60, orthogres, m=:o,
    xaxis=(L"m"),yaxis=(:log10,L"\| Q_m^TQ_m - I \|"), 
    title="Is Q orthonormal?",leg=:none)

In [ ]:
condQ = [cond(Q[:,1:m]) for m = 1:61]

In [ ]:
plot(0:60, condQ, m=:o,
    xaxis=(L"m"),yaxis=(:log10,L"\kappa(Q_m)"), 
    title="Is Q well conditioned?",leg=:none)

The Arnoldi bases are used to solve the least squares problems defining the GMRES iterates. 

In [ ]:
resid = [norm(b);zeros(60)]
for m = 1:60  
    s = [norm(b); zeros(m)]
    z = H[1:m+1,1:m]\s
    x = Q[:,1:m]*z
    resid[m+1] = norm(b-A*x)
 end

The approximations converge smoothly, practically all the way to machine epsilon.

In [ ]:
plot(0:60,resid,m=:o,
    xaxis=(L"m"),yaxis=(:log10,L"\| b-Ax_m \|"), 
    title="Residual for GMRES",leg=:none)

In [ ]:
resid[end]

# Example 8.5.2

The following experiments are based on a matrix resulting from discretization of a partial differential equation.

In [ ]:
d = 50;
A = d^2*matrixdepot("poisson",d)
n = size(A,1)

We compare unrestarted GMRES with three different thresholds for restarting.

### IterativeSolvers.jl

For the following test, we are using the `gmres` function from the [IterativeSolvers.jl](https://github.com/JuliaMath/IterativeSolvers.jl) package.

In [ ]:
methods(gmres)

In [ ]:
?gmres

In [ ]:
?gmres!

In [ ]:
b = ones(n);
maxit = 120;  rtol = 1e-8;
rest = [maxit,20,40,60]
plt = plot([],[],label="",title="Convergence of restarted GMRES",leg=:bottomleft,
    xaxis=("m"), yaxis=(:log10,"residual norm",[1e-8,100]))
for j = 1:4
    x,hist = gmres(A,b,restart=rest[j],tol=rtol,maxiter=maxit,log=true)
    plot!(hist[:resnorm],label="restart = $(rest[j])")
end
display(plt)

The "pure" curve is the lowest one. All of the other curves agree with it until they encounter their first restart. 

# Example 8.6.1

In this example we compare MINRES and CG on some pseudorandom SPD problems.  The first matrix has a condition number of 100. 

In [ ]:
n = 2000
density = 0.005
A = sprandsym(n,density,1e-2)
nnz(A)

We cook up a linear system whose solution we happen to know exactly.

In [ ]:
x = (1:n)/n
b = A*x;

Now we apply both methods and compare the convergence of the system residuals, using the built-in function `pcg` in the latter case.

In [ ]:
xMR,histMR = minres(A,b,tol=1e-12,maxiter=101,log=true)
xCG,histCG = cg(A,b,tol=1e-12,maxiter=101,log=true)

plot(0:100,[histMR[:resnorm] histCG[:resnorm]]/norm(b),m=:o,label=["MINRES" "CG"], 
    title="Convergence of MINRES and CG",
    xaxis=("Krylov dimension \$m\$"), yaxis=(:log10,L"\|r_m\| / \|b\|") )

There is virtually no difference between the two methods here when measuring the residual. We see little difference in the errors as well. 

In [ ]:
@show errorMR = norm( xMR - x ) / norm(x);
@show errorCG = norm( xCG - x) / norm(x);

Next we use a system matrix whose condition number is $10^4$. 

In [ ]:
A = sprandsym(n,density,1e-4);

Now we find that the CG residual jumps unexpectedly, but overall both methods converge at about the same linear rate. Note from the scales that both methods have actually made very little progress after 100 iterations, though. 

In [ ]:
xMR,histMR = minres(A,b,tol=1e-12,maxiter=101,log=true)
xCG,histCG = cg(A,b,tol=1e-12,maxiter=101,log=true)

plot(0:100,[histMR[:resnorm] histCG[:resnorm]]/norm(b),m=:o,label=["MINRES" "CG"], 
    title="Convergence of MINRES and CG",
    xaxis=("Krylov dimension \$m\$"), yaxis=(:log10,L"\|r_m\| / \|b\|") )

The errors confirm that we are nowhere near the correct solution in either case.

In [ ]:
@show errorMR = norm( xMR - x ) / norm(x);
@show errorCG = norm( xCG - x) / norm(x);

# Example 8.7.1

We use a readily available test image.

In [ ]:
img = testimage("mandrill");
@show m,n = size(img)
plot(Gray.(img),title="Original image",aspect_ratio=1)

We define the one-dimensional tridiagonal blurring matrices.

In [ ]:
X = @. Float64(Gray(img))

B = spdiagm(0=>fill(0.5,m),1=>fill(0.25,m-1),-1=>fill(0.25,m-1));
C = spdiagm(0=>fill(0.5,n),1=>fill(0.25,n-1),-1=>fill(0.25,n-1));

Finally, we show the results of using $k=12$ repetitions of the blur in each direction.

In [ ]:
blur = X -> B^12 * X * C^12;
plot(Gray.(blur(X)),title="Blurred image",aspect_ratio=1)

# Example 8.7.2

We repeat the earlier process to blur the original image $X$ to get $Z$. 

In [ ]:
img = testimage("mandrill");
m,n = size(img)

X = @. Float64(Gray(img))

B = spdiagm(0=>fill(0.5,m),1=>fill(0.25,m-1),-1=>fill(0.25,m-1));
C = spdiagm(0=>fill(0.5,n),1=>fill(0.25,n-1),-1=>fill(0.25,n-1));
blur = X -> B^12 * X * C^12;
Z = blur(X);

Now we imagine that $X$ is unknown and that the blurred $Z$ is given. We want to invert the blur transformation using the transformation itself. But we have to translate between vectors and images each time. 

In [ ]:
unvec = z -> reshape(z,m,n)
T = LinearMap(x -> vec(blur(unvec(x))),m*n);

Now we apply `gmres` to the composite blurring transformation `T`.

In [ ]:
y = gmres(T,vec(Z),maxiter=50,tol=1e-5);
Y = unvec(@.max(0,min(1,y)));

In [ ]:
plot(Gray.(X),layout=(1,3),subplot=1,title="Original",aspect_ratio=1,size=(900,300))
plot!(Gray.(Z),subplot=2,title="Blurred image",aspect_ratio=1)
plot!(Gray.(Y),subplot=3,title="Deblurred",aspect_ratio=1)

The reconstruction isn't perfect because the condition number of repeated blurring happens to be very large. 

# Example 8.8.1

Here is a large sparse matrix.

In [ ]:
A = 2.8I + sprand(10000,10000,0.002);

Without a preconditioner, GMRES takes a large number of iterations. 

In [ ]:
b = rand(10000)
gmres(A,b,maxiter=300,tol=1e-10,restart=50,log=true);
time_plain = @elapsed x,hist1 = gmres(A,b,maxiter=300,tol=1e-10,restart=50,log=true)

In [ ]:
plot(hist1[:resnorm],label="", 
    title="GMRES with no preconditioning",
    xaxis=("iteration number"), yaxis=(:log10,"residual norm") )

This version of incomplete $LU$ factorization drops all sufficiently small values (i.e., replaces them with zeros). This keeps the number of nonzeros in the factors under control.

In [ ]:
iLU = ilu(A,τ=0.2);
@show nnz(A),nnz(iLU.L);

It does _not_ produce a true factorization of $A$. However, it's close enough to serve as "approximate inverse" in a preconditioner. 

The actual preconditioning matrix is $\mathbf{M}=\mathbf{L}\mathbf{U}$. However, we just supply the factorization as a left preconditioner, since the preconditioning step is to solve a system with the matrix $\mathbf{M}$.

In [ ]:
time_prec = @elapsed x,hist2 = gmres(A,b,Pl=iLU,maxiter=300,tol=1e-10,restart=50,log=true)

The preconditioning is fairly successful in this case.

In [ ]:
plot(hist1[:resnorm],label="no prec.", 
    xaxis=("iteration number"), yaxis=(:log10,"residual norm") )
plot!(hist2[:resnorm],label="iLU prec.")

We probably made each GMRES iteration slower because of the need to apply the preconditioner (here, by solving sparse triangular systems). However, there are a lot fewer iterations needed, and there is a modest gain overall.

# Example 8.8.2

First we create a large, sparse, positive definite matrix that arises in the solution of differntial equations.

In [ ]:
n = 100
D2 = spdiagm(0=>fill(2,n-1),1=>-ones(n-2),-1=>-ones(n-2))
IA = spdiagm(0=>ones(n-1))
A = kron(IA,D2) + kron(D2,IA);

Now we solve a linear system with a random right-hand side, without preconditioner.

In [ ]:
b = rand(size(A,1))
cg(A,b,maxiter=4,tol=1e-10,log=true);  # make timing more accurate
time_plain = @elapsed x,hist1 = cg(A,b,maxiter=400,tol=1e-10,log=true)

In [ ]:
plot(hist1[:resnorm],label="", 
    title="CG with no preconditioning",
    xaxis=("iteration number"), yaxis=(:log10,"residual norm") )

For an SPD matrix we can use an incomplete Cholesky factorization. (It uses a lower triangular $\mathbf{L}=\mathbf{R}^T$ rather than an upper triangular $\mathbf{R}$.) 

In [ ]:
P = CholeskyPreconditioner(A,1);

Now we apply CG using this preconditioner. 

In [ ]:
cg(A,b,Pl=P,maxiter=4,tol=1e-10,log=true);  # make timing more accurate
time_prec = @elapsed x,hist2 = cg(A,b,Pl=P,maxiter=400,tol=1e-10,log=true)

In [ ]:
plot(hist1[:resnorm],label="no prec.", 
    xaxis=("iteration number"), yaxis=(:log10,"residual norm"),
    title="CG with incomplete Cholesky preconditioning")
plot!(hist2[:resnorm],label="iChol prec.")

You can see we got a little improvement with this preconditioner. It saved enough iterations to more than make up for the fact that each iteration now involves extra work.